In [5]:
import acquire
import prep

SyntaxError: invalid syntax (prep.py, line 1)

In [2]:
df = acquire.wrangle_zillow(acquire.get_sql_zillow())

In [4]:
df.T

,77578,77577,77576,77575,77394,77388,77389,77390,77391,77392,...,73,72,71,70,1,7,6,3,2,0
parcelid,12826780,12773139,17239384,11000655,11388980,12247692,11148930,11579452,11704525,12620810,...,12871444,11023776,10808296,11018202,17052889,12069064,12095076,12177905,14186244,14297519
basementsqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bathroomcnt,2,1,2,2,2,2,3,6,2,3,...,2,1,3,1,1,1,3,3,2,3.5
bedroomcnt,3,3,4,2,3,3,5,5,6,4,...,3,2,3,2,2,2,4,4,3,4
buildingqualitytypeid,6,4,NaN,6,6,6,7,11,6,8,...,4,4,8,4,NaN,5,9,8,NaN,NaN
calculatedbathnbr,2,1,2,2,2,2,3,6,2,3,...,2,1,3,1,1,1,3,3,2,3.5
decktypeid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
finishedfloor1squarefeet,NaN,NaN,1612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1465,NaN,NaN,NaN,NaN,NaN
calculatedfinishedsquarefeet,1762,1032,1612,1286,1518,1456,3208,5280,2475,2500,...,1059,776,2406,812,1465,738,2962,2376,1243,3100
finishedsquarefeet12,1762,1032,1612,1286,1518,1456,3208,5280,2475,2500,...,1059,776,2406,812,1465,738,2962,2376,1243,3100


In [8]:
df.longitude.isnull().sum()

0

In [9]:
df.groupby("regionidcounty")[["regionidcounty","regionidcity"]].count()

,regionidcounty,regionidcity
regionidcounty,,
1286.0,14110,13786
2061.0,4382,4369
3101.0,33800,33101
